In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import yaml
import os
from docx import Document
from nltk.tokenize import word_tokenize, sent_tokenize

from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings import GigaChatEmbeddings

from custom_modules.chunking_and_embeddings import GetEmbsNPY_Gigachat
from custom_modules.qdrant_database import PostEmbsQdrant
from custom_modules.retriever import Talk2DB

#import nltk
#nltk.download('punkt_tab')

In [ ]:
## global variables $ configs
config_path = r"configs\config.yml"
texts_path = r"laws"
config_path = r"configs\config.yml"

with open(config_path) as fh:
    read_config = yaml.load(fh, Loader=yaml.FullLoader)
giga_embeder_kwargs = read_config['EMBDER_CONFIG']
giga_chat_kwargs = read_config['LLM_CONFIG']

### 0 Word documents upload

In [ ]:
# Extract text content from .docx files
def getText(filename):
    doc = Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

# Load all .docx files from the folder 
def read_txt_files(directory):
    data = []
    for filename in tqdm(os.listdir(directory)):
        if filename.endswith('.docx'):
            text_path = os.path.join(texts_path, filename)
            text = getText(text_path)
            data.append([filename, text])
    return pd.DataFrame(data, columns=['filename', 'text'])

In [ ]:
# Read .docx files from the directory and save them to a Parquet file
df = read_txt_files(texts_path)

df.to_parquet(os.path.join(r'data/','texts.parquet'))

  0%|          | 0/16 [00:00<?, ?it/s]

### 1 basic text analysis

In [ ]:
# Analyze sentence lengths after additionally splitting by '\n'
df = pd.read_parquet('data/texts.parquet')
text_data = df['text'].tolist()

all_sentences = []
lengths = []
flattened_sentences = []
for text, i in zip(text_data, range(len(text_data))):
    flattened_sentences = []
    sentences = sent_tokenize(text) 
    sentences = [flattened_sentences.extend(sent.split('\n')) for sent in sentences] # делим по '\n'
    flattened_sentences = [sent for sent in flattened_sentences if len(sent)>0]
    all_sentences.extend(flattened_sentences)
    lengths.extend([len(word_tokenize(sentence)) for sentence in flattened_sentences])

# The longest sentence
print(max(lengths))
all_sentences[np.argmax(lengths)]

232


'В случае если выпуск (программа) облигаций дополнительно идентифицирован (идентифицирована) с использованием слов "зеленые облигации", "социальные облигации", "облигации устойчивого развития", "облигации, связанные с целями устойчивого развития", "адаптационные облигации" (далее соответственно - зеленые облигации, социальные облигации, облигации устойчивого развития, облигации, связанные с целями устойчивого развития, адаптационные облигации), информация о выпуске таких облигаций на этапе их размещения должна быть также раскрыта эмитентом в форме сообщения о проведении и результатах независимой внешней оценки соответствия выпуска (программы) указанных облигаций либо политики эмитента по использованию денежных средств, полученных от размещения указанных облигаций, или проекта (проектов), на финансирование (рефинансирование) которого (которых) будут использованы денежные средства, полученные от размещения указанных облигаций, принципам и стандартам финансовых инструментов, указанным в р

In [ ]:
# Total number of tokens across all texts
sum([len(word_tokenize(senten)) for senten in all_sentences])

622870

### 2 Obtain text chunks and build their embeddings

In [ ]:
text_data = df['text'].to_list()
text_chunks = []

for text, i in tqdm(list(zip(text_data, range(len(text_data))))):
    print(i)
    model = GetEmbsNPY_Gigachat([text], giga_embeder_kwargs, chunk_max_token_size = 150, overlap_ratio=0.15) # [text] чтобы чанкал по предложениям, иначе будет бить по буквам
    cur_embs, cur_text_chunks, cur_ids = model({})
    text_chunks.extend(cur_text_chunks)
    
    l = [cur_text_chunks, cur_ids]
    # save
    #pd.DataFrame(list(map(list, zip(*l))), columns=['data', 'ids']).to_parquet(f'data/chunks/chunks_{i+1}.parquet')
    #np.save(f'data/embs/embs_{i+1}.npy', cur_embs)


  0%|          | 0/16 [00:00<?, ?it/s]

0
Увеличьте максимальное количество токенов в 1 чанке. Длина куска текста 162 токенов
Увеличьте максимальное количество токенов в 1 чанке. Длина куска текста 154 токенов
1
2
Увеличьте максимальное количество токенов в 1 чанке. Длина куска текста 171 токенов
3
Увеличьте максимальное количество токенов в 1 чанке. Длина куска текста 161 токенов
Увеличьте максимальное количество токенов в 1 чанке. Длина куска текста 151 токенов
Увеличьте максимальное количество токенов в 1 чанке. Длина куска текста 155 токенов
Увеличьте максимальное количество токенов в 1 чанке. Длина куска текста 184 токенов
Увеличьте максимальное количество токенов в 1 чанке. Длина куска текста 155 токенов
Увеличьте максимальное количество токенов в 1 чанке. Длина куска текста 181 токенов
4
Увеличьте максимальное количество токенов в 1 чанке. Длина куска текста 156 токенов
5
6
7
8
Увеличьте максимальное количество токенов в 1 чанке. Длина куска текста 157 токенов
9
Увеличьте максимальное количество токенов в 1 чанке. Дли

In [ ]:
# check chunks length
lengths = []
for text in text_chunks:
    lengths.append(len(word_tokenize(text)))

print(max(lengths))
text_chunks[np.argmax(max(lengths))]

263


' Документ предоставлен КонсультантПлюс РОССИЙСКАЯ ФЕДЕРАЦИЯ ФЕДЕРАЛЬНЫЙ ЗАКОН О ПРОТИВОДЕЙСТВИИ НЕПРАВОМЕРНОМУ ИСПОЛЬЗОВАНИЮ ИНСАЙДЕРСКОЙ ИНФОРМАЦИИ И МАНИПУЛИРОВАНИЮ РЫНКОМ И О ВНЕСЕНИИ ИЗМЕНЕНИЙ В ОТДЕЛЬНЫЕ ЗАКОНОДАТЕЛЬНЫЕ АКТЫ РОССИЙСКОЙ ФЕДЕРАЦИИ Принят Государственной Думой 2 июля 2010 года Одобрен Советом Федерации 14 июля 2010 года Глава 1. ОБЩИЕ ПОЛОЖЕНИЯ Статья 1. Цель и сфера регулирования настоящего Федерального закона 1. Целью настоящего Федерального закона является обеспечение справедливого ценообразования на финансовые инструменты, иностранную валюту и (или) товары, равенства инвесторов и укрепление доверия инвесторов путем создания правового механизма предотвращения, выявления и пресечения злоупотреблений на организованных торгах в форме неправомерного использования инсайдерской информации и (или) манипулирования рынком. 2.'

In [ ]:
# Count total tokens (should exceed the original text because of overlapping chunks
text_chunks = [chunks for chunks in text_chunks if len(chunks)>0]
sum([len(word_tokenize(text)) for text in text_chunks])

756690

### 3 Store computed embeddings in the Qdrant vector database

In [ ]:
# Upload data to Qdrant in a loop (batch-wise)

def send_to_qdrant(input_path = r'data', collection_name = 'moex'):
    texts_list = os.listdir(input_path + '/chunks')
    embs_list = os.listdir(input_path + '/embs')

    for text_name, emb_name in tqdm(list(zip(texts_list, embs_list))): 
        global df_cur;
        df_cur = pd.read_parquet(input_path + '/chunks/' + text_name)
        df_cur = df_cur.rename({'index': 'ids'}, axis = 'columns')

        text_data = df_cur['data'].tolist() # texts
        metadata_ids = df_cur['ids'].tolist() # the rest info

        PostEmbsQdrant(input_path + '/embs/' + emb_name, 
                       text_data=text_data,
                       metadata_ids=metadata_ids,
                       collection_name=collection_name,
                       client_url='http://localhost:6333')()

In [ ]:
# Send to Qdrant
send_to_qdrant(input_path = r'data', collection_name = 'moex')

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Файлы успешно добавлены в базу, примерно 2112 строк наблюдений влезет в 25 мегабайт


  0%|          | 0/1 [00:00<?, ?it/s]

Файлы успешно добавлены в базу, примерно 2783 строк наблюдений влезет в 25 мегабайт


...

Файлы успешно добавлены в базу, примерно 575 строк наблюдений влезет в 25 мегабайт


  0%|          | 0/1 [00:00<?, ?it/s]

Файлы успешно добавлены в базу, примерно 175 строк наблюдений влезет в 25 мегабайт


  0%|          | 0/1 [00:00<?, ?it/s]

Файлы успешно добавлены в базу, примерно 2058 строк наблюдений влезет в 25 мегабайт


### 4 Retriever

In [ ]:
# Set up the assistant instance

# Initialize the embedder for the user query (Gigachat)
embedder_inited_cls = GigaChatEmbeddings(**giga_embeder_kwargs)

talk2db = Talk2DB(collection_name=read_config['DB_CONFIG']['collection_name'],
                  embedder_inited_cls=embedder_inited_cls,
                  payload_key=read_config['DB_CONFIG']['payload_key'],
                  query_prompt=read_config['PROMPT_CONFIG']['query_prompt'],
                  response_prompt=read_config['PROMPT_CONFIG']['response_prompt'],
                  num_queries=int(read_config['PROMPT_CONFIG']['num_queries']),
                  max_context_length=int(read_config['PROMPT_CONFIG']['max_context_length']))

llm = GigaChat(max_tokens = 500,
                temperature = 1.2,
               **giga_chat_kwargs)

## TEST

In [ ]:
# Execute the query and inspect the returned result
# Ensure that the Qdrant server is up and running
answer, retrieved_chunks, original_query, formulated_query = await talk2db.get_response('Какими бумагами можно торговать на моосковской бирже?', llm, history = [])

c:\Programming\venvs\telegram_news\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
answer, retrieved_chunks, original_query, formulated_query

('Ценные бумаги иностранного эмитента, соответствующие определенным требованиям, могут быть допущены к организованным торгам на Московской бирже без заключения договора с эмитентом. Однако, если эти ценные бумаги не были допущены к публичному размещению и обращению в РФ, они должны соответствовать требованиям и ограничениям, установленным законодательством для ценных бумаг, предназначенных для квалифицированных инвесторов. Кроме того, Банк России имеет право определять основные списки иностранных бирж, включение в которые является условием для допуска ценных бумаг к торговле на Московской бирже [Контекст].',
 [' Ценные бумаги иностранного эмитента, соответствующие требованиям пунктов 1 и 2 настоящей статьи, могут быть допущены к публичному обращению в Российской Федерации по решению российского организатора торговли об их допуске к организованным торгам без заключения договора с указанным эмитентом, если ценные бумаги иностранного эмитента одновременно соответствуют следующим условиям:

In [ ]:
# Demonstration with conversation history included
history = [('сколько всего нормативных актов на мосбирже', 'согласно последней документации - 16'), ('какая средняя длина данных документов', 'в среднем один документ состоит из 15 страниц'), ('содержится ли информация в них о евробондах','')]
print("\n".join([f"User: {user_msg}\nBot: {bot_resp}" for user_msg, bot_resp in history if bot_resp]))

answer, context, original_query, formulated_query = await talk2db.get_response('Какими бумагами можно торговать на моосковской бирже?', llm, history = history)
answer, context, original_query, formulated_query

User: сколько всего нормативных актов на мосбирже
Bot: согласно последней документации - 16
User: какая средняя длина данных документов
Bot: в среднем один документ состоит из 15 страниц


('На Московской бирже можно торговать различными видами ценных бумаг, включая акции, облигации, паи инвестиционных фондов, депозитарные расписки и другие финансовые инструменты.',
 [' Ценные бумаги иностранного эмитента, соответствующие требованиям пунктов 1 и 2 настоящей статьи, могут быть допущены к публичному обращению в Российской Федерации по решению российского организатора торговли об их допуске к организованным торгам без заключения договора с указанным эмитентом, если ценные бумаги иностранного эмитента одновременно соответствуют следующим условиям: 1) допускаются к организованным торгам без их включения в котировальный список; 2) начата или завершена процедура включения в основной (официальный) список ценных бумаг иностранной биржи, соответствующей критериям, указанным в пункте 4 настоящей статьи. Банк России вправе определить основные (официальные) списки иностранных бирж, включение в которые является условием для допуска ценных бумаг к публичному обращению в Российской Феде